In [14]:
from sklearn.datasets import make_blobs
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
import pandas
from imblearn.over_sampling import SMOTE
import scipy.io
import numpy as np
import pandas as pd

In [15]:
data = scipy.io.loadmat('x.mat')  
print(data['x'].shape) 

(2042, 105, 79)


In [16]:
columns = np.asarray([['Electrode %d - %d/2 Hz'%(i+1, j+1)] for i in range(data['x'].shape[1]) for j in range(data['x'].shape[2])])
columns

array([['Electrode 1 - 1/2 Hz'],
       ['Electrode 1 - 2/2 Hz'],
       ['Electrode 1 - 3/2 Hz'],
       ...,
       ['Electrode 105 - 77/2 Hz'],
       ['Electrode 105 - 78/2 Hz'],
       ['Electrode 105 - 79/2 Hz']], dtype='<U23')

In [17]:
labels = pd.read_csv("table_withlabels.csv")
foof = pd.read_csv("foof2features.csv")

,IDs,Intercept,Slope,Unnamed: 0,Age,Sex,ACE_Score,APQ_P_Total,APQ_SR_Total,ARI_P_Total_Score,...,DX_10_Past_Doc,DX_10_Presum,DX_10_RC,DX_10_Rem,DX_10_RuleOut,DX_10_Spec,DX_10_Sub,DX_10_Time,NoDX,label
0,NDARAA075AMK,0.986272,1.825774,1,6.728040,1,NaN,91.0,123.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,No Diagnosis Given
1,NDARAA112DMH,1.486650,1.888544,2,5.545744,0,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,ADHD-Combined Type
2,NDARAA117NEJ,1.593155,2.095749,3,7.475929,0,NaN,113.0,95.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,ADHD-Combined Type
3,NDARAA947ZG5,0.703331,1.724831,7,13.627880,0,NaN,85.0,88.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,ADHD-Combined Type
4,NDARAA948VFH,0.918020,1.749441,8,7.982660,1,NaN,91.0,91.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,ADHD-Combined Type
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,NDARZN148PMN,0.168009,0.205704,3028,11.629363,1,NaN,102.0,145.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,Anxiety Disorders
1781,NDARZN277NR6,1.351549,1.996940,3030,14.878736,1,NaN,104.0,112.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,ADHD-Combined Type
1782,NDARZN610GTY,0.339229,1.050644,3031,16.379534,1,NaN,106.0,114.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,Other Neurodevelopmental Disorders
1783,NDARZN677EYE,0.781225,1.470061,3032,15.029545,1,NaN,105.0,121.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,ADHD-Inattentive Type


In [18]:
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df.columns = columns
df['IDs'] = foof['C1']
df = pd.merge(df, labels[['label', 'IDs']], on='IDs', how='inner')
print(df['label'].value_counts())
dataset = df.values
print(dataset.shape)


Other Neurodevelopmental Disorders    492
ADHD-Inattentive Type                 388
ADHD-Combined Type                    376
Anxiety Disorders                     241
No Diagnosis Given                    203
Depressive Disorders                   85
Name: label, dtype: int64
(1785, 8297)


In [19]:
X = dataset[:,0:8295].astype(float)
y = dataset[:,8296]




# scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
#X = np.clip(X, -1, 1)

X.shape

(1785, 8295)

In [20]:

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
y = np_utils.to_categorical(y)

print(y)

[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [21]:
from sklearn.model_selection import train_test_split
trainX, testX, trainy, testy = train_test_split(X, y, stratify=y,test_size=0.3)

oversample = SMOTE() #oversample to aaccount for the data imbalance
trainX,trainy = oversample.fit_resample(trainX,trainy)
testX,testy = oversample.fit_resample(testX,testy)

print(trainX.shape)
testX.shape

(2064, 8295)


(888, 8295)

In [22]:


trainX = trainX.reshape((2064,105,79)) 
testX = testX.reshape((888,105,79)) 



In [23]:


from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def evaluate(y_test, y_pred, show_cm=False):
    print(y_test)
    print(y_pred)
    y_test = np.argmax(y_test, axis=1) # assuming you have n-by-6 class_prob
    y_pred = np.argmax(y_pred, axis=1) # assuming you have n-by-6 class_prob
    print(y_test)
    print(y_pred)
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    print("Precision:", metrics.precision_score(y_test, y_pred, average='macro'))
    print("Recall:", metrics.recall_score(y_test, y_pred, average='macro'))
#     print("ROC AUC:", metrics.roc_auc_score(y_test, y_pred, multi_class='ovo',))
    print("F1 score:", metrics.f1_score(y_test, y_pred, average='macro'))
#     print("Brier Score:", metrics.brier_score_loss(y_test, y_pred)) # only for binary classification
    labels = ['Other Neurodevelopmental Disorders', 'ADHD-Inattentive Type', 'ADHD-Combined Type', 'Anxiety Disorders', 'No Diagnosis Given', 'Depressive Disorders']
    if show_cm:
        cm = confusion_matrix(y_test, y_pred, labels=labels)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        disp.plot()
        plt.tight_layout()
        plt.show()



In [24]:
trainX = np.transpose(trainX, (0, 2,1))
testX = np.transpose(testX, (0, 2,1))


In [48]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
print('Tensorflow Version: ',tensorflow.__version__)
from tensorflow.keras.layers import BatchNormalization

Tensorflow Version:  2.2.0


In [79]:
# Initialising the CNN
model = Sequential()
# Convolution
model.add(Conv1D(filters = 32, kernel_size =4, input_shape = (79, 105), activation = 'relu', padding = 'same'))
#model.add(Conv1D(filters = 32, kernel_size = 5, input_shape = (79, 32), activation = 'relu', padding = 'same'))
# Pooling
model.add(MaxPooling1D(pool_size = 2))
# Convolution
model.add(Conv1D(filters = 64, kernel_size = 3, input_shape = (39, 32), activation = 'relu', padding = 'same'))
##model.add(Conv1D(filters = 64, kernel_size = 3, input_shape = (39, 64), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size = 2))
model.add(Conv1D(filters =128, kernel_size = 3, input_shape = (19, 64), activation = 'relu', padding = 'same'))
##model.add(Conv1D(filters =128, kernel_size = 3, input_shape = (19, 128), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size = 2))
model.add(Conv1D(filters =256, kernel_size = 3, input_shape = (9, 128), activation = 'relu', padding = 'same'))
#model.add(Conv1D(filters =256, kernel_size = 3, input_shape = (9, 256), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size = 2))
model.add(Conv1D(filters =512, kernel_size = 3, input_shape = (4, 256), activation = 'relu', padding = 'same'))
#model.add(Conv1D(filters =512, kernel_size = 3, input_shape = (4, 512), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size = 2))
model.add(Conv1D(filters = 1024, kernel_size = 3, input_shape = (2, 512), activation = 'relu', padding = 'same'))
#model.add(Conv1D(filters = 1024, kernel_size = 3, input_shape = (2, 1028), activation = 'relu', padding = 'same'))
model.add(MaxPooling1D(pool_size = 2))
# Flattening
model.add(Flatten())
# Full connection
#model.add(Dense(units = 512, activation = 'relu'))
#model.add(Dropout(0.5))
#model.add(Dense(units = 128, activation = 'relu'))
# Add Dropout to prevent overfitting
#model.add(Dropout(0.5))
#model.add(Dense(units = 32, activation = 'relu'))
#model.add(Dropout(0.5))
model.add(Dense(units = 6, activation = 'softmax'))
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_158 (Conv1D)          (None, 79, 32)            13472     
_________________________________________________________________
max_pooling1d_143 (MaxPoolin (None, 39, 32)            0         
_________________________________________________________________
conv1d_159 (Conv1D)          (None, 39, 64)            6208      
_________________________________________________________________
batch_normalization_60 (Batc (None, 39, 64)            256       
_________________________________________________________________
max_pooling1d_144 (MaxPoolin (None, 19, 64)            0         
_________________________________________________________________
conv1d_160 (Conv1D)          (None, 19, 128)           24704     
_________________________________________________________________
batch_normalization_61 (Batc (None, 19, 128)         

In [88]:
# Initialising the CNN
model = Sequential()
# Convolution
model.add(Conv1D(filters = 32, kernel_size = 3, input_shape = (79, 105), activation = 'relu', padding = 'same'))
# Pooling
model.add(MaxPooling1D(pool_size = 2))
# Convolution
model.add(Conv1D(filters = 64, kernel_size = 3, input_shape = (39, 32), activation = 'relu', padding = 'same'))
model.add(Conv1D(filters = 64, kernel_size = 3, input_shape = (39, 64), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size = 2))
model.add(Conv1D(filters =128, kernel_size = 3, input_shape = (19, 64), activation = 'relu', padding = 'same'))
##model.add(Conv1D(filters =128, kernel_size = 3, input_shape = (19, 128), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size = 2))
model.add(Conv1D(filters =256, kernel_size = 3, input_shape = (9, 128), activation = 'relu', padding = 'same'))
#model.add(Conv1D(filters =256, kernel_size = 3, input_shape = (9, 256), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size = 2))
model.add(Conv1D(filters =512, kernel_size = 3, input_shape = (4, 256), activation = 'relu', padding = 'same'))
#model.add(Conv1D(filters =512, kernel_size = 3, input_shape = (4, 512), activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size = 2))
model.add(Conv1D(filters = 1024, kernel_size = 3, input_shape = (2, 512), activation = 'relu', padding = 'same'))
model.add(Conv1D(filters = 1024, kernel_size = 3, input_shape = (2, 1028), activation = 'relu', padding = 'same'))
model.add(MaxPooling1D(pool_size = 2))
# Flattening
model.add(Flatten())

model.add(Dense(units = 6, activation = 'softmax'))
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_185 (Conv1D)          (None, 79, 32)            10112     
_________________________________________________________________
max_pooling1d_167 (MaxPoolin (None, 39, 32)            0         
_________________________________________________________________
conv1d_186 (Conv1D)          (None, 39, 64)            6208      
_________________________________________________________________
conv1d_187 (Conv1D)          (None, 39, 64)            12352     
_________________________________________________________________
batch_normalization_76 (Batc (None, 39, 64)            256       
_________________________________________________________________
max_pooling1d_168 (MaxPoolin (None, 19, 64)            0         
_________________________________________________________________
conv1d_188 (Conv1D)          (None, 19, 128)         

In [89]:
history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=20, verbose=1)
# evaluate the model
evaluate(testy, model.predict(testX))#, show_cm=True)
#evaluate(trainy, model.predict(trainX))#, show_cm=True)


Epoch 1/20
65/65 [==============================] - 1s 14ms/step - loss: 1.8801 - accuracy: 0.3193 - val_loss: 2.5084 - val_accuracy: 0.1847
Epoch 2/20
65/65 [==============================] - 1s 10ms/step - loss: 1.3107 - accuracy: 0.4797 - val_loss: 2.4738 - val_accuracy: 0.2252
Epoch 3/20
65/65 [==============================] - 1s 9ms/step - loss: 1.1593 - accuracy: 0.5383 - val_loss: 2.3377 - val_accuracy: 0.2376
Epoch 4/20
65/65 [==============================] - 1s 10ms/step - loss: 0.9040 - accuracy: 0.6579 - val_loss: 2.9332 - val_accuracy: 0.1959
Epoch 5/20
65/65 [==============================] - 1s 10ms/step - loss: 0.7676 - accuracy: 0.7035 - val_loss: 3.0121 - val_accuracy: 0.2140
Epoch 6/20
65/65 [==============================] - 1s 10ms/step - loss: 0.6397 - accuracy: 0.7616 - val_loss: 3.3537 - val_accuracy: 0.1768
Epoch 7/20
65/65 [==============================] - 1s 10ms/step - loss: 0.5314 - accuracy: 0.8261 - val_loss: 2.9276 - val_accuracy: 0.1903
Epoch 8/20
65/